In [101]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from imblearn.over_sampling import SMOTE

In [102]:
df=pd.read_csv('grant_data_imb.csv')
pd.set_option('display.max_columns',None)

In [103]:
df.sample()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,SEO.Code.1,SEO.Percentage.1,SEO.Code.2,SEO.Percentage.2,SEO.Code.3,SEO.Percentage.3,SEO.Code.4,SEO.Percentage.4,SEO.Code.5,SEO.Percentage.5,Person.ID.1,Role.1,Year.of.Birth.1,Country.of.Birth.1,Home.Language.1,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
3631,0,18B,20A,G,321216.0,60.0,321206.0,20.0,321204.0,20.0,0.0,0.0,0.0,0.0,730217.0,40.0,730219.0,40.0,730213.0,20.0,0.0,0.0,0.0,0.0,84622.0,CHIEF_INVESTIGATOR,1955.0,Australia,NaN,2973.0,25.0,NaN,>=0 to 5,0.0,5.0,8.0,10.0,2.0,5.0


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Grant.Status                            4113 non-null   int64  
 1   Sponsor.Code                            3856 non-null   object 
 2   Grant.Category.Code                     3856 non-null   object 
 3   Contract.Value.Band...see.note.A        1953 non-null   object 
 4   RFCD.Code.1                             3853 non-null   float64
 5   RFCD.Percentage.1                       3853 non-null   float64
 6   RFCD.Code.2                             3853 non-null   float64
 7   RFCD.Percentage.2                       3853 non-null   float64
 8   RFCD.Code.3                             3853 non-null   float64
 9   RFCD.Percentage.3                       3853 non-null   float64
 10  RFCD.Code.4                             3853 non-null   floa

In [105]:
target = df['Grant.Status']
features = df.drop(columns=['Grant.Status'])

In [106]:
target.value_counts()

Grant.Status
0    3259
1     854
Name: count, dtype: int64

In [107]:
def upsample(features, target, repaet=10):
    features_zeros=features[target==0]
    features_ones=features[target==1]
    target_zeros=target[target==0]
    target_ones=target[target==1]
    if len(target_ones)>len(target_zeros):
        repaet=round(len(target_ones)/len(target_zeros))
        features_upsampled=pd.concat([features_ones]+ [features_zeros]*repaet)
        target_upsampled=pd.concat([target_ones]+[target_zeros]*repaet)
    else:
        repaet=round(len(target_zeros)/len(target_ones))
        features_upsampled=pd.concat([features_zeros]+ [features_ones]*repaet)
        target_upsampled=pd.concat([target_zeros]+[target_ones]*repaet)
    features_upsampled, target_upsampled=shuffle(features_upsampled, target_upsampled, random_state=23)
    return features_upsampled, target_upsampled


In [108]:
features_train_upsampled, target_train_upsampled=upsample(features, target)

In [109]:
numerical_features = features.select_dtypes(include=['float64', 'int64']).columns

# Заполнение пропусков средними значениями и нулями
for feature in numerical_features:
    features[feature + '_mean'] = features[feature].fillna(features[feature].mean())
    features[feature + '_zero'] = features[feature].fillna(0)

In [110]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 98 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            3856 non-null   object 
 1   Grant.Category.Code                     3856 non-null   object 
 2   Contract.Value.Band...see.note.A        1953 non-null   object 
 3   RFCD.Code.1                             3853 non-null   float64
 4   RFCD.Percentage.1                       3853 non-null   float64
 5   RFCD.Code.2                             3853 non-null   float64
 6   RFCD.Percentage.2                       3853 non-null   float64
 7   RFCD.Code.3                             3853 non-null   float64
 8   RFCD.Percentage.3                       3853 non-null   float64
 9   RFCD.Code.4                             3853 non-null   float64
 10  RFCD.Percentage.4                       3853 non-null   floa

In [111]:
features['With.PHD.1'].fillna('No', inplace=True)
features

C:\Users\korsh624\AppData\Local\Temp\ipykernel_2908\1439241994.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  features['With.PHD.1'].fillna('No', inplace=True)


,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,SEO.Code.1,SEO.Percentage.1,SEO.Code.2,SEO.Percentage.2,SEO.Code.3,SEO.Percentage.3,SEO.Code.4,SEO.Percentage.4,SEO.Code.5,SEO.Percentage.5,Person.ID.1,Role.1,Year.of.Birth.1,Country.of.Birth.1,Home.Language.1,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1,RFCD.Code.1_mean,RFCD.Code.1_zero,RFCD.Percentage.1_mean,RFCD.Percentage.1_zero,RFCD.Code.2_mean,RFCD.Code.2_zero,RFCD.Percentage.2_mean,RFCD.Percentage.2_zero,RFCD.Code.3_mean,RFCD.Code.3_zero,RFCD.Percentage.3_mean,RFCD.Percentage.3_zero,RFCD.Code.4_mean,RFCD.Code.4_zero,RFCD.Percentage.4_mean,RFCD.Percentage.4_zero,RFCD.Code.5_mean,RFCD.Code.5_zero,RFCD.Percentage.5_mean,RFCD.Percentage.5_zero,SEO.Code.1_mean,SEO.Code.1_zero,SEO.Percentage.1_mean,SEO.Percentage.1_zero,SEO.Code.2_mean,SEO.Code.2_zero,SEO.Percentage.2_mean,SEO.Percentage.2_zero,SEO.Code.3_mean,SEO.Code.3_zero,SEO.Percentage.3_mean,SEO.Percentage.3_zero,SEO.Code.4_mean,SEO.Code.4_zero,SEO.Percentage.4_mean,SEO.Percentage.4_zero,SEO.Code.5_mean,SEO.Code.5_zero,SEO.Percentage.5_mean,SEO.Percentage.5_zero,Person.ID.1_mean,Person.ID.1_zero,Year.of.Birth.1_mean,Year.of.Birth.1_zero,Dept.No..1_mean,Dept.No..1_zero,Faculty.No..1_mean,Faculty.No..1_zero,Number.of.Successful.Grant.1_mean,Number.of.Successful.Grant.1_zero,Number.of.Unsuccessful.Grant.1_mean,Number.of.Unsuccessful.Grant.1_zero,A..1_mean,A..1_zero,A.1_mean,A.1_zero,B.1_mean,B.1_zero,C.1_mean,C.1_zero
0,97A,30B,A,321024.0,50.0,321013.0,30.0,291502.0,20.0,0.0,0.0,0.0,0.0,730303.0,50.0,730104.0,30.0,671402.0,20.0,0.0,0.0,0.0,0.0,67037.0,CHIEF_INVESTIGATOR,1950.0,Eastern Europe,NaN,2563.0,25.0,Yes,>10 to 15,2.0,6.0,3.0,5.0,15.0,3.0,321024.000000,321024.0,50.00000,50.0,321013.000000,321013.0,30.000000,30.0,291502.000000,291502.0,20.000000,20.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,730303.00000,730303.0,50.000000,50.0,730104.00000,730104.0,30.000000,30.0,671402.000000,671402.0,20.000000,20.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,67037.0,67037.0,1950.0,1950.0,2563.0,2563.0,25.0,25.0,2.0,2.0,6.0,6.0,3.0,3.0,5.0,5.0,15.0,15.0,3.0,3.0
1,36D,10A,G,300201.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,620108.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12917.0,PRINCIPAL_SUPERVISOR,1955.0,Australia,NaN,1038.0,1.0,No,Less than 0,0.0,3.0,0.0,4.0,0.0,0.0,300201.000000,300201.0,100.00000,100.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,620108.00000,620108.0,100.000000,100.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,12917.0,12917.0,1955.0,1955.0,1038.0,1038.0,1.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0
2,317A,30D,NaN,321013.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730104.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28607.0,CHIEF_INVESTIGATOR,1965.0,Great Britain,NaN,2763.0,25.0,Yes,>5 to 10,4.0,3.0,6.0,25.0,14.0,14.0,321013.000000,321013.0,100.00000,100.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,730104.00000,730104.0,100.000000,100.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,28607.0,28607.0,1965.0,1965.0,2763.0,2763.0,25.0,25.0,4.0,4.0,3.0,3.0,6.0,6.0,25.0,25.0,14.0,14.0,14.0,14.0
3,62B,10B,B,321103.0,30.0,321105.0,40.0,321204.0,30.0,0.0,0.0,0.0,0.0,730211.0,50.0,730302.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,144632.0,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,2848.0,25.0,No,Less than 0,1.0,2.0,1.0,0.0,0.0,0.0,321103.000000,321103.0,30.00000,30.0,321105.000000,321105.0,40.000000,40.0,321204.000000,321204.0,30.000000,30.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,730211.00000,730211.0,50.000000,

In [112]:
for col in features.columns:
    if (features[col].dtype=='object'):
        features[col].fillna('not indicated', inplace=True)

C:\Users\korsh624\AppData\Local\Temp\ipykernel_2908\4064073959.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  features[col].fillna('not indicated', inplace=True)


In [113]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 98 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            4113 non-null   object 
 1   Grant.Category.Code                     4113 non-null   object 
 2   Contract.Value.Band...see.note.A        4113 non-null   object 
 3   RFCD.Code.1                             3853 non-null   float64
 4   RFCD.Percentage.1                       3853 non-null   float64
 5   RFCD.Code.2                             3853 non-null   float64
 6   RFCD.Percentage.2                       3853 non-null   float64
 7   RFCD.Code.3                             3853 non-null   float64
 8   RFCD.Percentage.3                       3853 non-null   float64
 9   RFCD.Code.4                             3853 non-null   float64
 10  RFCD.Percentage.4                       3853 non-null   floa

In [114]:
features_one=pd.get_dummies(features, drop_first=True)
features_one.sample()

,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,SEO.Code.1,SEO.Percentage.1,SEO.Code.2,SEO.Percentage.2,SEO.Code.3,SEO.Percentage.3,SEO.Code.4,SEO.Percentage.4,SEO.Code.5,SEO.Percentage.5,Person.ID.1,Year.of.Birth.1,Dept.No..1,Faculty.No..1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1,RFCD.Code.1_mean,RFCD.Code.1_zero,RFCD.Percentage.1_mean,RFCD.Percentage.1_zero,RFCD.Code.2_mean,RFCD.Code.2_zero,RFCD.Percentage.2_mean,RFCD.Percentage.2_zero,RFCD.Code.3_mean,RFCD.Code.3_zero,RFCD.Percentage.3_mean,RFCD.Percentage.3_zero,RFCD.Code.4_mean,RFCD.Code.4_zero,RFCD.Percentage.4_mean,RFCD.Percentage.4_zero,RFCD.Code.5_mean,RFCD.Code.5_zero,RFCD.Percentage.5_mean,RFCD.Percentage.5_zero,SEO.Code.1_mean,SEO.Code.1_zero,SEO.Percentage.1_mean,SEO.Percentage.1_zero,SEO.Code.2_mean,SEO.Code.2_zero,SEO.Percentage.2_mean,SEO.Percentage.2_zero,SEO.Code.3_mean,SEO.Code.3_zero,SEO.Percentage.3_mean,SEO.Percentage.3_zero,SEO.Code.4_mean,SEO.Code.4_zero,SEO.Percentage.4_mean,SEO.Percentage.4_zero,SEO.Code.5_mean,SEO.Code.5_zero,SEO.Percentage.5_mean,SEO.Percentage.5_zero,Person.ID.1_mean,Person.ID.1_zero,Year.of.Birth.1_mean,Year.of.Birth.1_zero,Dept.No..1_mean,Dept.No..1_zero,Faculty.No..1_mean,Faculty.No..1_zero,Number.of.Successful.Grant.1_mean,Number.of.Successful.Grant.1_zero,Number.of.Unsuccessful.Grant.1_mean,Number.of.Unsuccessful.Grant.1_zero,A..1_mean,A..1_zero,A.1_mean,A.1_zero,B.1_mean,B.1_zero,C.1_mean,C.1_zero,Sponsor.Code_101A,Sponsor.Code_103C,Sponsor.Code_105A,Sponsor.Code_107C,Sponsor.Code_111C,Sponsor.Code_112D,Sponsor.Code_113A,Sponsor.Code_11C,Sponsor.Code_120D,Sponsor.Code_126B,Sponsor.Code_128D,Sponsor.Code_12D,Sponsor.Code_130B,Sponsor.Code_132D,Sponsor.Code_133A,Sponsor.Code_135C,Sponsor.Code_136D,Sponsor.Code_137A,Sponsor.Code_138B,Sponsor.Code_139C,Sponsor.Code_13A,Sponsor.Code_141A,Sponsor.Code_143C,Sponsor.Code_144D,Sponsor.Code_145A,Sponsor.Code_146B,Sponsor.Code_147C,Sponsor.Code_148D,Sponsor.Code_149A,Sponsor.Code_14B,Sponsor.Code_150B,Sponsor.Code_151C,Sponsor.Code_153A,Sponsor.Code_154B,Sponsor.Code_156D,Sponsor.Code_157A,Sponsor.Code_158B,Sponsor.Code_159C,Sponsor.Code_15C,Sponsor.Code_160D,Sponsor.Code_161A,Sponsor.Code_163C,Sponsor.Code_164D,Sponsor.Code_165A,Sponsor.Code_166B,Sponsor.Code_168D,Sponsor.Code_169A,Sponsor.Code_16D,Sponsor.Code_170B,Sponsor.Code_172D,Sponsor.Code_173A,Sponsor.Code_174B,Sponsor.Code_176D,Sponsor.Code_177A,Sponsor.Code_178B,Sponsor.Code_179C,Sponsor.Code_180D,Sponsor.Code_183C,Sponsor.Code_184D,Sponsor.Code_187C,Sponsor.Code_188D,Sponsor.Code_18B,Sponsor.Code_191C,Sponsor.Code_193A,Sponsor.Code_194B,Sponsor.Code_195C,Sponsor.Code_196D,Sponsor.Code_197A,Sponsor.Code_198B,Sponsor.Code_199C,Sponsor.Code_1A,Sponsor.Code_200D,Sponsor.Code_201A,Sponsor.Code_202B,Sponsor.Code_203C,Sponsor.Code_204D,Sponsor.Code_205A,Sponsor.Code_206B,Sponsor.Code_208D,Sponsor.Code_20D,Sponsor.Code_210B,Sponsor.Code_212D,Sponsor.Code_214B,Sponsor.Code_215C,Sponsor.Code_216D,Sponsor.Code_219C,Sponsor.Code_21A,Sponsor.Code_221A,Sponsor.Code_222B,Sponsor.Code_223C,Sponsor.Code_225A,Sponsor.Code_226B,Sponsor.Code_227C,Sponsor.Code_228D,Sponsor.Code_229A,Sponsor.Code_230B,Sponsor.Code_232D,Sponsor.Code_234B,Sponsor.Code_235C,Sponsor.Code_238B,Sponsor.Code_23C,Sponsor.Code_241A,Sponsor.Code_242B,Sponsor.Code_244D,Sponsor.Code_245A,Sponsor.Code_247C,Sponsor.Code_24D,Sponsor.Code_250B,Sponsor.Code_252D,Sponsor.Code_255C,Sponsor.Code_256D,Sponsor.Code_257A,Sponsor.Code_258B,Sponsor.Code_259C,Sponsor.Code_260D,Sponsor.Code_262B,Sponsor.Code_265A,Sponsor.Code_266B,Sponsor.Code_267C,Sponsor.Code_269A,Sponsor.Code_26B,Sponsor.Code_270B,Sponsor.Code_273A,Sponsor.Code_274B,Sponsor.Code_277A,Sponsor.Code_279C,Sponsor.Code_27C,Sponsor.Code_281A,Sponsor.Code_282B,Sponsor.Code_284D,Sponsor.Code_285A,Sponsor.Code_286B,Sponsor.Code_289A,Sponsor.Code_28D,Sponsor.Code_294B,Sponsor.Code_295C,Sponsor

In [115]:
features_train, features_test, target_train, target_test = train_test_split(features_one, target, test_size=0.2, random_state=42)


In [116]:
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

In [117]:
pd.DataFrame(features_test_scaled,columns=features_train.columns).dropna(inplace=True)

In [118]:
model=LogisticRegressionCV(solver='liblinear',random_state=12,class_weight='balanced',cv=10)
model.fit(features_train_scaled, target_train)
roc_auc_score(target_test, model.predict_proba(features_test_scaled))

ValueError: Input X contains NaN.
LogisticRegressionCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values